## Compare Runs

- https://mlflow.org/docs/latest/getting-started/quickstart-2/index.html

### Init mlflow

- Run `uv tool run mlflow ui`

In [22]:
import mlflow
mlflow.set_experiment('compare-runs')
mlflow.set_tracking_uri('http://127.0.0.1:5000')

2024/12/24 09:50:12 INFO mlflow.tracking.fluent: Experiment with name 'compare-runs' does not exist. Creating a new experiment.


### Hyperparameter sweep

In [23]:
import keras
import numpy as np
import pandas as pd
from hyperopt import STATUS_OK, Trials, fmin, hp, tpe
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

import mlflow
from mlflow.models import infer_signature

In [24]:
# Load dataset
data = pd.read_csv(
    "https://raw.githubusercontent.com/mlflow/mlflow/master/tests/datasets/winequality-white.csv",
    sep=";",
)

# Split the data into training, validation, and test sets
train, test = train_test_split(data, test_size=0.25, random_state=42)
train_x = train.drop(["quality"], axis=1).values
train_y = train[["quality"]].values.ravel()
test_x = test.drop(["quality"], axis=1).values
test_y = test[["quality"]].values.ravel()
train_x, valid_x, train_y, valid_y = train_test_split(
    train_x, train_y, test_size=0.2, random_state=42
)
signature = infer_signature(train_x, train_y)

In [25]:
def train_model(params, epochs, train_x, train_y, valid_x, valid_y, test_x, test_y):
    # Define model architecture
    mean = np.mean(train_x, axis=0)
    var = np.var(train_x, axis=0)
    model = keras.Sequential(
        [
            keras.Input([train_x.shape[1]]),
            keras.layers.Normalization(mean=mean, variance=var),
            keras.layers.Dense(64, activation="relu"),
            keras.layers.Dense(1),
        ]
    )

    # Compile model
    model.compile(
        optimizer=keras.optimizers.SGD(
            learning_rate=params["lr"], momentum=params["momentum"]
        ),
        loss="mean_squared_error",
        metrics=[keras.metrics.RootMeanSquaredError()],
    )

    # Train model with MLflow tracking
    with mlflow.start_run(nested=True):
        model.fit(
            train_x,
            train_y,
            validation_data=(valid_x, valid_y),
            epochs=epochs,
            batch_size=64,
        )
        # Evaluate the model
        eval_result = model.evaluate(valid_x, valid_y, batch_size=64)
        eval_rmse = eval_result[1]

        # Log parameters and results
        mlflow.log_params(params)
        mlflow.log_metric("eval_rmse", eval_rmse)

        # Log model
        mlflow.tensorflow.log_model(model, "model", signature=signature)

        return {"loss": eval_rmse, "status": STATUS_OK, "model": model}


In [26]:
def objective(params):
    # MLflow will track the parameters and results for each run
    result = train_model(
        params,
        epochs=3,
        train_x=train_x,
        train_y=train_y,
        valid_x=valid_x,
        valid_y=valid_y,
        test_x=test_x,
        test_y=test_y,
    )
    return result

In [27]:
space = {
    "lr": hp.loguniform("lr", np.log(1e-5), np.log(1e-1)),
    "momentum": hp.uniform("momentum", 0.0, 1.0),
}

In [28]:
with mlflow.start_run():
    # Conduct the hyperparameter search using Hyperopt
    trials = Trials()
    best = fmin(
        fn=objective,
        space=space,
        algo=tpe.suggest,
        max_evals=8,
        trials=trials,
    )

    # Fetch the details of the best run
    best_run = sorted(trials.results, key=lambda x: x["loss"])[0]

    # Log the best parameters, loss, and model
    mlflow.log_params(best)
    mlflow.log_metric("eval_rmse", best_run["loss"])
    mlflow.tensorflow.log_model(best_run["model"], "model", signature=signature)

    # Print out the best parameters and corresponding loss
    print(f"Best parameters: {best}")
    print(f"Best eval rmse: {best_run['loss']}")


  0%|          | 0/8 [00:00<?, ?trial/s, best loss=?]

Epoch 1/3                                            

 1/46 ━━━━━━━━━━━━━━━━━━━━ 9s 221ms/step - loss: 36.4182 - root_mean_squared_error: 6.0348
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 35.0780 - root_mean_squared_error: 5.9226 - val_loss: 34.3121 - val_root_mean_squared_error: 5.8577

Epoch 2/3                                            

 1/46 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - loss: 33.3675 - root_mean_squared_error: 5.7765
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 33.7298 - root_mean_squared_error: 5.8077 - val_loss: 33.0668 - val_root_mean_squared_error: 5.7504

Epoch 3/3                                            

 1/46 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 33.3176 - root_mean_squared_error: 5.7721
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 32.5289 - root_mean_squared_error: 5.7034 - val_loss: 31.8744 - val_root_mean_squared_error: 5.6457

 1/12 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 31.9091 - root_mean_squared_error: 5.6488
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 

2024/12/24 09:50:24 WARNING mlflow.utils.environment: Failed to resolve installed pip version. ``pip`` will be added to conda.yaml environment spec without a version specifier.



🏃 View run big-panda-325 at: http://127.0.0.1:5000/#/experiments/586931892202866225/runs/34a6298370ca4b8d88a6d18fba3b710c

🧪 View experiment at: http://127.0.0.1:5000/#/experiments/586931892202866225

Epoch 1/3                                                                     

 1/46 ━━━━━━━━━━━━━━━━━━━━ 8s 191ms/step - loss: 32.2449 - root_mean_squared_error: 5.6785
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 8.4269 - root_mean_squared_error: 2.8160 - val_loss: 1.1654 - val_root_mean_squared_error: 1.0795

Epoch 2/3                                                                     

 1/46 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 1.0376 - root_mean_squared_error: 1.0186
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 1.2450 - root_mean_squared_error: 1.1142 - val_loss: 0.8255 - val_root_mean_squared_error: 0.9086

Epoch 3/3                                                                     

 1/46 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - loss: 0.8368 - root_mean_squared_error: 0.914

2024/12/24 09:50:29 WARNING mlflow.utils.environment: Failed to resolve installed pip version. ``pip`` will be added to conda.yaml environment spec without a version specifier.



...

🏃 View run adaptable-duck-443 at: http://127.0.0.1:5000/#/experiments/586931892202866225/runs/dd65b9d8a5644ee39d854475115f1483

🧪 View experiment at: http://127.0.0.1:5000/#/experiments/586931892202866225  

Epoch 1/3                                                                      

 1/46 ━━━━━━━━━━━━━━━━━━━━ 23s 520ms/step - loss: 35.9804 - root_mean_squared_error: 5.9984
46/46 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 33.8188 - root_mean_squared_error: 5.8132 - val_loss: 24.5562 - val_root_mean_squared_error: 4.9554

Epoch 2/3                                                                      

 1/46 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - loss: 23.0981 - root_mean_squared_error: 4.8060
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 22.0688 - root_mean_squared_error: 4.6956 - val_loss: 15.8639 - val_root_mean_squared_error: 3.9830

Epoch 3/3                                                                      

 1/46 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - loss: 15.6048 - root_mean_squ

2024/12/24 09:51:02 WARNING mlflow.utils.environment: Failed to resolve installed pip version. ``pip`` will be added to conda.yaml environment spec without a version specifier.



🏃 View run luminous-tern-672 at: http://127.0.0.1:5000/#/experiments/586931892202866225/runs/a15ba48792b240b582eb2822afe501f7

🧪 View experiment at: http://127.0.0.1:5000/#/experiments/586931892202866225  

100%|██████████| 8/8 [00:43<00:00,  5.46s/trial, best loss: 0.9010204076766968]


2024/12/24 09:51:07 WARNING mlflow.utils.environment: Failed to resolve installed pip version. ``pip`` will be added to conda.yaml environment spec without a version specifier.


Best parameters: {'lr': np.float64(0.07589263848913844), 'momentum': np.float64(0.9108498774937469)}
Best eval rmse: 0.9010204076766968
🏃 View run unleashed-ram-478 at: http://127.0.0.1:5000/#/experiments/586931892202866225/runs/4630bcae703d403ea406b5b38afa356b
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/586931892202866225


### Compare results

- UI based, see web page for walkthrough

### Register and deploy best model

- Click on the best run from your charts, and there is a "Register Model" in the UI. Input the name of your deployment model
- Go to `Models` tab, and add metadata as needed
- Deploy using `mlflow models serve -m "models:/compare-runs/1" --port 5002`
    - If this fails, check that `virtualenv` is installed

In [31]:
# !mlflow models serve -m "models:/compare-runs/1" --port 5002

- Test model deployment using curl

    ```bash
    curl -d '{"dataframe_split": {
    "columns": ["fixed acidity","volatile acidity","citric acid","residual sugar","chlorides","free sulfur dioxide","total sulfur dioxide","density","pH","sulphates","alcohol"],
    "data": [[7,0.27,0.36,20.7,0.045,45,170,1.001,3,0.45,8.8]]}}' \
    -H 'Content-Type: application/json' -X POST localhost:5002/invocations
    ```

    ```bash
    {"predictions": [[5.535236358642578]]}
    ```

### Build container for model

- Mlflow also lets you build a docker image for your models automatically!

- If this fails
    - check that docker daemon is running
    - You may be connecting to a different socket, if you are using colima, change `DOCKER_HOST` environment variable to the right path:
        - `export DOCKER_HOST=unix:///Users/$(whoami)/.colima/docker.sock`

- To deploy the built container, mlflow is integrated with the major cloud providers. See link for detailed deployment steps

- `mlflow models build-docker --model-uri "models:/compare-runs/1" --name "qs_mlops"`
- `docker run -p 5002:8080 qs_mlops`

In [34]:
# !mlflow models build-docker --model-uri "models:/compare-runs/1" --name "qs_mlops"
# !docker run -p 5002:8080 qs_mlops